In [1]:
import os
import re
import pandas as pd
import sqlite3
import configparser
import shlex
import subprocess
import json
import atexit
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from typing import TypedDict, Dict, Any

# Определяем тип состояния
class GraphState(TypedDict):
    input: str
    vtune_file_path: str
    export_result: str
    result: str

# Глобальные переменные
last_exported_csv_path = None
gradio_interface_instance = None

# Чтение API-ключа
check_path = os.path.dirname(os.getcwd())
config_path = os.path.join(check_path, "GPTPROF", "config.ini")
config = configparser.ConfigParser()
config.read(config_path)

try:
    mistral_api_key = config["API_KEYS"]["MISTRAL_API_KEY"]
except KeyError:
    raise ValueError("API key not found in config.ini")

# Инициализация модели
mistral_model = "codestral-latest"
llm = ChatMistralAI(model=mistral_model, temperature=0, api_key=mistral_api_key)

In [2]:
@tool
def export_vtune_to_csv(vtune_file_path: str) -> str:
    """
    Exports data from a .vtune file to a CSV file using Intel VTune.
    Returns path to exported CSV or error message.
    """
    global last_exported_csv_path

    if not vtune_file_path:
        return "Error: File path is empty"

    vtune_file_path = vtune_file_path.strip('"')
    
    if not vtune_file_path.endswith(".vtune"):
        return "Error: File must have .vtune extension"

    if not os.path.exists(vtune_file_path):
        return f"Error: File {vtune_file_path} not found"

    try:
        file_name = os.path.splitext(os.path.basename(vtune_file_path))[0]
        output_csv = f"{file_name}.csv"
        command = f'vtune -report hotspots -r {shlex.quote(vtune_file_path)} -format=csv -report-output {output_csv}'

        subprocess.run(command, shell=True, check=True)
        last_exported_csv_path = output_csv
        return f"Data exported to {output_csv}"

    except subprocess.CalledProcessError as e:
        return f"Export failed: {e}"

# Адаптер для узла графа
def export_vtune_node(state: dict) -> dict:
    """
    Node that processes VTune export.
    Exports data from a .vtune file to CSV and returns updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state with the result of the export and next node.
    """
    vtune_path = state.get("vtune_file_path", "")
    tool_result = export_vtune_to_csv(vtune_path)

    # Очищаем состояние от параметров других узлов
    return {
        "input": "",
        "export_result": tool_result,
        "last_exported_csv": last_exported_csv_path if "Data exported to" in tool_result else None,
        "next_node": "decision"
    }

In [3]:
@tool
def filter_by_module(module_names: list, csv_file_path: str = None) -> str:
    """
    Filters data by specified modules and saves the result in last_exported_csv.

    Args:
        module_names (list): List of module names to filter by.
        csv_file_path (str, optional): Path to the CSV file for filtering. 
                                       If not provided, uses last_exported_csv_path.

    Returns:
        str: Message indicating the result of the operation.
    """
    global last_exported_csv_path

    # Проверяем, что передан хотя бы один модуль для фильтрации
    if not module_names:
        return "Error: No modules specified for filtering."

    # Определяем путь к CSV-файлу
    if csv_file_path:
        if not os.path.exists(csv_file_path):
            return f"Error: File {csv_file_path} not found."
        file_to_filter = csv_file_path
    elif last_exported_csv_path:
        file_to_filter = last_exported_csv_path
    else:
        return "Error: No data available for filtering. Export data from .vtune first."

    try:
        # Читаем данные из CSV-файла
        df = pd.read_csv(file_to_filter, sep="\t")

        # Фильтруем данные по модулям и исходным файлам
        filtered_df = df[
            (df["Module"].isin(module_names)) &
            (df["Source File"].str.endswith(".cpp") | df["Source File"].str.endswith(".hpp"))
            ]

        # Если после фильтрации данных нет
        if filtered_df.empty:
            return "Error: No data found for the specified modules."

        # Формируем имя для отфильтрованного файла
        file_name = os.path.basename(file_to_filter)
        filtered_csv_path = os.path.join(os.path.dirname(file_to_filter), f"filtered_{file_name}")

        filtered_df.to_csv(filtered_csv_path, sep="\t", index=False)
        last_exported_csv_path = filtered_csv_path

        return f"Data filtered by modules {module_names} and saved to {filtered_csv_path}."
    except Exception as e:
        return f"Error processing data: {e}"


# Адаптер для узла графа
def filter_by_module_node(state: dict) -> dict:
    """
    Node adapter for the filter_by_module tool.
    Extracts module names and CSV file path from the state, calls the tool,
    and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after filtering.
    """
    # Извлекаем параметры из состояния
    module_names = state.get("module_names", [])
    csv_file_path = state.get("csv_file_path", None)

    # Вызываем тул для фильтрации данных
    tool_result = filter_by_module(module_names, csv_file_path)

    # Возвращаем обновленное состояние
    return {
        "input": "",
        "filter_result": tool_result,
        "last_exported_csv": last_exported_csv_path,
        "next_node": "decision"
    }

In [4]:
# Узел принятия решений
def decide_next_step(state: dict) -> dict:
    """
    Decision node that determines the next step based on user input.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state with the next node and result.
    """
    user_input = state.get("input", "").lower().strip()

    if "export" in user_input:
        vtune_path = user_input.replace("export", "").strip()
        if not vtune_path:
            return {
                "next_node": "decision",
                "result": "Error: Please provide a valid .vtune file path",
                **state
            }
        return {
            "vtune_file_path": vtune_path,
            "next_node": "export_vtune",
            **state
        }
    elif "filter" in user_input:
        parts = user_input.split("filter", 1)
        if len(parts) < 2:
            return {
                "next_node": "decision",
                "result": "Error: Please provide module names for filtering.",
                **state
            }
        module_names = [name.strip() for name in parts[1].split(",") if name.strip()]
        return {
            "module_names": module_names,
            "next_node": "filter_by_module",
            **state
        }
    elif user_input == "exit":
        return {
            "next_node": "end"
        }
    else:
        return {
            "next_node": "decision",
            "result": "Unknown command. Available: export <path>, filter <module1,module2,...> or exit",
            **state
        }

def end_node(state: dict) -> dict:
    """
    End node to finalize the workflow.
    This node can be used to display a goodbye message or perform cleanup tasks.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Final state with a termination message.
    """
    return {
        "result": "Session terminated. Thank you for using the tool!",
        "next_node": END
    }
    
# Инициализация графа
graph = StateGraph(GraphState)

# Добавляем узлы
graph.add_node("decision", decide_next_step)
graph.add_node("end", end_node)
graph.add_node("export_vtune", export_vtune_node)
graph.add_node("filter_by_module", filter_by_module_node)

# Настройка переходов
graph.add_edge("decision", "export_vtune")
graph.add_edge("decision", "filter_by_module")
graph.add_edge("decision", "end")
graph.add_edge("export_vtune", "decision")
graph.add_edge("filter_by_module", "decision")

graph.set_entry_point("decision")
compiled_graph = graph.compile()

In [ ]:
# Обработчик запросов
def ask_codestral(user_input: str, session_state: dict = None):
    state = session_state or {"input": user_input}
    try:
        result_state = compiled_graph.invoke(state)

        # Принудительное завершение при команде exit
        if result_state.get("input", "").lower().strip() == "exit":
            if gradio_interface_instance is not None:
                gradio_interface_instance.close()
            os._exit(0)

        return result_state.get("export_result", result_state.get("result")), result_state
    except Exception as e:
        return f"Error: {str(e)}", None

def gradio_interface(user_input: str, session_state: dict = None):
    return ask_codestral(user_input, session_state)

def launch_gradio():
    global gradio_interface_instance

    if gradio_interface_instance is not None:
        gradio_interface_instance.close()

    with gr.Blocks() as interface:
        with gr.Row():
            cmd_input = gr.Textbox(label="Command",
                                   placeholder="Enter your response",
                                   interactive=True)
            session_state = gr.State()
            output = gr.Textbox(label="Output",
                                interactive=False,
                                lines=10,
                                autoscroll=True)

        submit_btn = gr.Button("Submit", variant="primary")

        def process_command(cmd: str, state: dict):
            response, new_state = gradio_interface(cmd, state)
            return response, new_state

        # Обработка нажатия кнопки и Enter в текстовом поле
        submit_btn.click(
            process_command,
            inputs=[cmd_input, session_state],
            outputs=[output, session_state]
        )
        cmd_input.submit(
            process_command,
            inputs=[cmd_input, session_state],
            outputs=[output, session_state]
        )

        # Инструкции для пользователя
        gr.Markdown("""
        ### Available commands:
        - `export <file.vtune>` - Export VTune data to CSV
        - `exit` - Terminate the session
        """)

    gradio_interface_instance = interface

    # Настройки запуска
    interface.launch(
        server_port=7860,
        inbrowser=False,
        share=False,
        show_error=True
    )

# Запускаем интерфейс
launch_gradio()